In [ ]:
#Imports

import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
!pip install evaluate
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
MODEL = "bert-base-uncased"
BATCH_SIZE = 4
EPOCHS = 10

In [ ]:
#Loading in train data

train = pd.read_csv("/kaggle/input/english-sarcasm-detection/train.En.csv")
train_C = pd.read_csv("/kaggle/input/english-sarcasm-detection/english_task_c.csv")
train_a = pd.read_csv("/kaggle/input/english-sarcasm-detection/english_task_a.csv")
train_B = pd.read_csv("/kaggle/input/english-sarcasm-detection/task_B_En_test.csv")

test = pd.read_csv("/kaggle/input/english-sarcasm-detection/task_A_En_test.csv")

In [ ]:
#Extracting parts of training data relevant for Task A

train_A = train.loc[:, ["tweet", "sarcastic", "rephrase"]]
train_C = train_C.loc[:, ["text_1", "text_0", "human_aggregated"]]
train_a = train_a.loc[:, ["text", "human_aggregated"]]
train_B = train_B.loc[:, ["text", "sarcasm", "irony", "satire", "understatement", "overstatement", "rhetorical_question"]]

test_A = test.loc[:, ["text", "sarcastic"]]

In [ ]:
#Conveting train data to a format suitable for training

transformed_train_A = []
transformed_train_B = []
transformed_train_C = []
transformed_train_a = []

for i in range(len(train_A)):
    transformed_train_A.append({
        "label": train_A.iloc[i]["sarcastic"],
        "text": train_A.iloc[i]["tweet"]
    })
    
    transformed_train_A.append({
        "label": 0,
        "text": train_A.iloc[i]["rephrase"]
    })

for i in range(len(train_B)):
    if train_B.iloc[i]["sarcasm"] == 1 or train_B.iloc[i]["irony"] == 1 or train_B.iloc[i]["satire"] == 1 or train_B.iloc[i]["understatement"] == 1 or train_B.iloc[i]["overstatement"] == 1 or train_B.iloc[i]["rhetorical_question"] == 1:
        transformed_train_B.append({
        "label": 1,
        "text": train_B.iloc[i]["text"]
    })
    else:
        transformed_train_B.append({
        "label": 0,
        "text": train_B.iloc[i]["text"]
    })
    
for i in range(len(train_C)):
    if train_C.iloc[i]["human_aggregated"] == 0:
        transformed_train_C.append({
            "label": 1,
            "text": train_C.iloc[i]["text_0"]
        })
        
        transformed_train_C.append({
            "label": 0,
            "text": train_C.iloc[i]["text_1"]
        })
    else:
        transformed_train_C.append({
            "label": 0,
            "text": train_C.iloc[i]["text_0"]
        })
        
        transformed_train_C.append({
            "label": 1,
            "text": train_C.iloc[i]["text_1"]
        })
    
for i in range(len(train_a)):
    transformed_train_a.append({
        "label": train_a.iloc[i]["human_aggregated"],
        "text": train_a.iloc[i]["text"]
    })
    
transformed_test_A = []

for i in range(len(test_A)):
    transformed_test_A.append({
        "label": test_A.iloc[i]["sarcastic"],
        "text": test_A.iloc[i]["text"]
    })

In [ ]:
#Tokenization

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def preprocess_function(text):
    return tokenizer(text, truncation=True)

tokenized_transformed_train_A = []

for dictionary in transformed_train_A:
    if type(dictionary["text"]) is type("string"):
        temp_dict = preprocess_function(dictionary["text"])
        temp_dict["label"] = dictionary["label"]
        tokenized_transformed_train_A.append(temp_dict)
        
tokenized_transformed_train_C = []

for dictionary in transformed_train_C:
    if type(dictionary["text"]) is type("string"):
        temp_dict = preprocess_function(dictionary["text"])
        temp_dict["label"] = dictionary["label"]
        tokenized_transformed_train_C.append(temp_dict)
        
tokenized_transformed_train_a = []

for dictionary in transformed_train_a:
    if type(dictionary["text"]) is type("string"):
        temp_dict = preprocess_function(dictionary["text"])
        temp_dict["label"] = dictionary["label"]
        tokenized_transformed_train_a.append(temp_dict)
        
tokenized_transformed_train_B = []

for dictionary in transformed_train_B:
    if type(dictionary["text"]) is type("string"):
        temp_dict = preprocess_function(dictionary["text"])
        temp_dict["label"] = dictionary["label"]
        tokenized_transformed_train_B.append(temp_dict)
        
tokenized_transformed_test_A = []

for dictionary in transformed_test_A:
    if type(dictionary["text"]) is type("string"):
        temp_dict = preprocess_function(dictionary["text"])
        temp_dict["label"] = dictionary["label"]
        tokenized_transformed_test_A.append(temp_dict)

In [ ]:
#Extending A dataset with community labelled C dataset

tokenized_transformed_train_A.extend(tokenized_transformed_train_a)
tokenized_transformed_train_A.extend(tokenized_transformed_train_C)

In [ ]:
#Dynamic batchwise padding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#Evaluation metrics

accuracy = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels, pos_label=1, average = "binary")

In [ ]:
#Id2label map and vice versa

id2label = {0: "non-sarcastic", 1: "sarcastic"}
label2id = {"non-sarcastic": 0, "sarcastic": 1}

In [ ]:
#Defining the model

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL, num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
#Training and evaluation

training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_transformed_train_A,
    eval_dataset=tokenized_transformed_test_A,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
#f461aa864bc375133e0abc309bda5598d66d00a2

In [ ]:
!rm -rf /kaggle/working/